In [357]:
import random
import string
from datetime import date, datetime, timedelta

MYSQL_CONFIG = {
    'host': 'mysql',
    'user': 'root',
    'port': 3307,
    'password': '',
    'database': 'itildesk1'
}

ACCOUNT_ID = 2

class DBConnection(object):
    
    def __enter__(self, *args, **kwargs):
        self.connection = pymysql.connect(
            **MYSQL_CONFIG,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )
        self.cursor = self.connection.cursor()
        return self
    
    def __exit__(self, *args, **kwargs):
        self.connection and self.connection.close()
        self.cursor and self.cursor.close()
        
    def commit(self):
        self.cursor.commit()

In [368]:
class CustomORM(object):
    def __new__(self, *args, **kwargs):
        table_name, fetch_type = (args[0], args[1])
        data = []
        col = args[2] if len(args) >= 3 else '*'
        with DBConnection() as db:
            print(f'Fetching::{table_name}::{fetch_type}::{col}')
            q = f'select {col} from `{table_name}` where account_id = {ACCOUNT_ID}'
            if fetch_type == 'last':
                q += f' order by {col} desc limit 1;'
            print('Executing Query\n\t{q}'.format(q = q))
            db.cursor.execute(q)
            data = db.cursor.fetchall()
            data = list(map(lambda x : x[col], data))
            if fetch_type == 'last' and len(data) > 0:
                data = data[0]
        return data

In [394]:
import string
from datetime import datetime

REF_ID_MAP = {}
ENTITY_MAP = {
    'department': ["Sales", "Marketing", "Finance", "Human Resources", "IT", "Operations"],
    'asset': ['Virtual Machine', 'AWS VM', 'Azure VM', 'VMware VCenter VM', 'Volume', 'AWS Disk', 'Azure Disk', 'VMware VCenter Disk', 'Host', 'VMware VCenter Host', 'Custom Asset Type'],
    'group': ['Incident Team', 'Major Incident Team', 'Service Request Fulfillment Team', 'Problem Management Team', 'Change Team', 'Release Team', 'Database Team', 'Hardware Team', 'Capacity Management Team', 'Supplier Management Team', 'Service Design Team', 'Software Team', 'Network Team', 'Helpdesk Monitoring Team'],
    'location': [],
    'name': ['John', 'Doe', 'Ray', 'fsTest']
}

def get_table_fields(table_name):
    with DBConnection() as db:
        db.cursor.execute(f'describe {table_name}')
        return list(map(lambda x : (x['Field'], x['Null']), db.cursor.fetchall()))

def get_random_string(_type, size = 10):
    if _type == 'number':
        return ''.join([secrets.choice(string.digits) for _ in range(size)])
    elif _type == 'alpha':
        characters = string.ascii_letters + string.digits
        return ''.join(secrets.choice(characters) for _ in range(size))
    elif _type == 'name':
        return random.choice(ENTITY_MAP['name']) + get_random_string("number", 5)
    elif _type == 'location':
        return f'Test Loc-{get_random_string("alpha", 5)}'
    else:
        return f'{random.choice(ENTITY_MAP[_type])}-{get_random_string("alpha", 5)}'

def get_cur_date():
    return datetime.now().isoformat()

def get_ref_id(ref_id):
    ref_id = str(ref_id)[0:3] + ''.join([random.choice(string.digits) for _ in range(12)])
    if ref_id in REF_ID_MAP:
        print('Got duplicate REF iD')
        return get_ref_id()
    REF_ID_MAP[ref_id] = True
    return ref_id

_types = ['number', 'alpha', 'department', 'location', 'group', 'name', 'asset']
for _type in _types:
    print(get_random_string(_type))

1189061875
zUJcDR6E6D
IT-i7Iik
Test Loc-bY7XN
Problem Management Team-nrD5b
Doe57880
Host-ODSF3


In [403]:
IMPACTS = ['low', 'medium', 'high']
FIELD_TYPES = ['text', 'paragraph', 'number', 'integer', 'dropdown', 'date', 'checkbox', 'bigint']
WORKSPACE_ID = 2
REF_ID = 110000000000000
SEED = {
    'locations': 100,
    'departments': 100,
    'asset_types': 100,
    'requesters': 100,
    'agents': 100,
    'assets': 100,
    'groups': 100,
    'type_fields': 100
}

class SeedData:
    def __init__(self, db):
        self.db = db
        
    def execute_query(self, query, params=None):
        self.db.cursor.execute(query, params)
        if params is None: return self.db.cursor.fetchall()
        
    def execute_many(self, query, params):
        self.db.cursor.executemany(query, params)
        self.db.connection.commit()
        
    def fetch_required_meta(self):
        self.asset_states = CustomORM('cmdb_ci_type_field_values', 'list', 'name')
        self.vendor_ids = CustomORM('itil_vendors', 'list', 'id')
        self.product_ids = CustomORM('itil_products', 'list', 'id')
        self.department_ids = CustomORM('itil_departments', 'list', 'id')
        self.location_ids = CustomORM('cmdb_locations', 'list', 'id')
        self.ref_ids = CustomORM('cmdb_ci_types', 'list', 'ref_id')
        self.group_ids = CustomORM('groups', 'list', 'id')
        
        query = f"""
        WITH RECURSIVE cte AS (
            SELECT id, name, parent_ci_type
            FROM cmdb_ci_types
            WHERE ref_id = {REF_ID} and account_id = {ACCOUNT_ID}
            UNION ALL
            SELECT t.id, t.name, t.parent_ci_type
            FROM cmdb_ci_types t
            JOIN cte c ON t.parent_ci_type = c.id
        )
        SELECT id, name, parent_ci_type
        FROM cte;
        """
        self.ci_type_ids = list(map(lambda x : x['id'], self.execute_query(query)))

    def seed_entity_data(self):
        self.fetch_required_meta()        
        self.seed_asset_types()
        self.seed_users(False)
        self.seed_departments()
        self.seed_locations()
        self.seed_users(True)
        self.seed_agent_groups()
        self.seed_type_fields()
        self.fetch_required_meta()
        self.seed_assets()
        
    def seed_asset_types(self):
        query = """
        INSERT INTO cmdb_ci_types (
            name, description, parent_ci_type, is_default, account_id, ancestry, ancestry_depth, 
            created_at, updated_at, ref_id, label, disabled
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        params = []
        for _ in range(SEED['asset_types']):
            params.append([
                get_random_string('asset'),
                "",
                self.ci_type_ids[0],
                False,
                ACCOUNT_ID,
                self.ci_type_ids[0],
                1,
                get_cur_date(),
                get_cur_date(),
                get_ref_id(REF_ID),
                get_random_string('asset'),
                0
            ])
        self.execute_many(query, params)

    def seed_locations(self):
        query = """
        INSERT INTO cmdb_locations (name, account_id)
        VALUES (%s, %s)
        """
        params = []
        for _ in range(SEED['locations']):
            params.append([
                get_random_string('location'),  # name
                ACCOUNT_ID
            ])
        self.execute_many(query, params)
        
    def seed_departments(self):
        query = """
        INSERT INTO itil_departments (name, description, account_id, head_id, prime_user_id, created_at, updated_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        params = []
        for _ in range(SEED['departments']):
            params.append([
                get_random_string('department'),
                "",
                ACCOUNT_ID,
                None,
                None,
                get_cur_date(),
                get_cur_date()
            ])
        self.execute_many(query, params)
        
    def seed_users(self, is_agent = False):
        query = """
        INSERT INTO users (name, email, persistence_token, login_count, failed_login_count, account_id, active, delta, helpdesk_agent, created_at, updated_at) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        params = []
        for _ in range(SEED['agents'] if is_agent else SEED['requesters']):
            params.append([
                get_random_string('name'),
                get_random_string('name') + '@gmail.com',
                "",
                0, 
                0,
                ACCOUNT_ID,
                1,
                0,
                1 if is_agent else 0,
                get_cur_date(),
                get_cur_date()
            ])
        last_user_id = CustomORM('users', 'last', 'id')
        self.execute_many(query, params)
        if is_agent:
            self.seed_agents(last_user_id)

    def seed_agents(self, last_user_id):
        query = """
        INSERT INTO agents (user_id, account_id, available, license_type) values (%s, %s, %s, %s);
        """
        params = []
        for _ in range(SEED['agents']):
            last_user_id += 1
            params.append([
                last_user_id,
                ACCOUNT_ID,
                1,
                1
            ])
        self.execute_many(query, params)
   
    def seed_agent_groups(self):
        query = """
        INSERT INTO itildesk1.groups (name, description, account_id, type, business_function, workspace_id) 
        VALUES (%s, %s, %s, %s, %s, %s);
        """
        params = []
        for _ in range(SEED['groups']):
            params.append([
                get_random_string('group'),
                "",
                ACCOUNT_ID,
                'Group',
                0,
                WORKSPACE_ID
            ])
        self.execute_many(query, params)

    def seed_type_fields(self):
        ci_type_id = self.ci_type_ids[0]
        last_position = CustomORM('cmdb_ci_type_fields', 'last', 'position')
        query = """
        INSERT INTO cmdb_ci_type_fields (
            name, label, active, field_type, position, required, table_name, column_name, ci_type_id, account_id, 
            col_type, deleted, is_unique
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        params = []
        for _ in range(SEED['type_fields']):
            last_position += 1
            field_type = random.choice(FIELD_TYPES)
            params.append([
                f'{field_type}_{get_random_string("alpha")}',
                get_random_string('asset'),
                1,
                f'custom_{field_type}',
                last_position,
                0,
                'cmdb_ci_level_0_fields',
                '',
                ci_type_id, ACCOUNT_ID, field_type, 0, 0
            ])
        self.execute_many(query, params)
        

    def seed_assets(self):
        query = """
        INSERT INTO cmdb_config_items (name, ci_type_id, impact, account_id, department_id, group_id, display_id, location_id, roots_ref_id)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        params = []
        for _ in range(SEED['assets']):
            params.append([
                get_random_string('asset'),  # name
                self.ci_type_ids[0],  # ci_type_id
                IMPACTS.index(random.choice(IMPACTS)),  # impact
                ACCOUNT_ID,
                random.choice(self.department_ids),
                random.choice(self.group_ids),
                get_random_string('number'),
                random.choice(self.location_ids),
                REF_ID
            ])
        self.execute_many(query, params)
        
with DBConnection() as db:
    seeder = SeedData(db)
    seeder.seed_entity_data()

Fetching::cmdb_ci_type_field_values::list::name
Executing Query
	select name from `cmdb_ci_type_field_values` where account_id = 2
Fetching::itil_vendors::list::id
Executing Query
	select id from `itil_vendors` where account_id = 2
Fetching::itil_products::list::id
Executing Query
	select id from `itil_products` where account_id = 2
Fetching::itil_departments::list::id
Executing Query
	select id from `itil_departments` where account_id = 2
Fetching::cmdb_locations::list::id
Executing Query
	select id from `cmdb_locations` where account_id = 2
Fetching::cmdb_ci_types::list::ref_id
Executing Query
	select ref_id from `cmdb_ci_types` where account_id = 2
Fetching::groups::list::id
Executing Query
	select id from `groups` where account_id = 2
Fetching::users::last::id
Executing Query
	select id from `users` where account_id = 2 order by id desc limit 1;
Fetching::users::last::id
Executing Query
	select id from `users` where account_id = 2 order by id desc limit 1;
Fetching::cmdb_ci_type_fi

In [404]:
import pymysql
import random
import string
import secrets
from datetime import datetime

MYSQL_CONFIG = {
	'host': '127.0.0.1',
	'user': 'shard_3_writer',
	'port': 6033,
	'password': 'Iou72GH#',
	'database': 'freshbnbs_shard2'
}
ACCOUNT_ID = 5302337
WORKSPACE_ID = 2
REF_ID = 110000000000000
REF_ID_MAP = {}
ENTITY_MAP = {
	'department': ["Sales", "Marketing", "Finance", "Human Resources", "Operations", "Customer Service", "Information Technology", "Research and Development", "Administration", "Product Management", "Quality Assurance", "Logistics", "Procurement", "Public Relations", "Legal", "Training and Development", "Accounting", "Business Development", "Project Management", "Communications", "Engineering", "Supply Chain", "Compliance", "Design", "Manufacturing", "Facilities Management", "Risk Management", "Data Analytics", "Strategic Planning", "Customer Success", "Event Management", "Internal Audit", "Talent Acquisition", "Health and Safety", "Sustainability", "Sales Operations", "Market Research", "Vendor Management", "Employee Relations", "Innovation", "Business Intelligence", "Partnerships", "Branding", "Inventory Control", "Training and Education", "Consulting", "Media Relations", "Financial Analysis", "Software Development", "Performance Management", "Compensation and Benefits", "Systems Administration", "Public Affairs", "Client Services", "E-commerce", "Database Administration", "Event Planning", "Network Engineering", "Social Media Management", "Web Development", "Internal Communications", "Budgeting and Forecasting", "Talent Management", "Product Marketing", "Outsourcing", "User Experience (UX)", "International Business", "Change Management", "Brand Management", "Crisis Management", "Digital Marketing", "Hardware Engineering", "Procurement and Contracts", "Knowledge Management", "Application Support", "Security Operations", "Content Development", "Market Analysis", "Business Process Improvement", "Software Testing", "Data Governance", "Performance Optimization", "Sales Training", "Compensation Management", "Incident Management", "Market Expansion", "Database Management", "Public Speaking", "Cloud Computing", "Financial Reporting", "Training Delivery", "UI/UX Design", "Vendor Relations", "Mobile App Development", "Workplace Culture", "Regulatory Compliance", "Product Design", "Customer Relationship Management (CRM)", "Customer Support", "Business Analysis"],
	'asset': ["AWS Disk", "AWS VM", "Azure Disk", "Azure VM", "VMware vCenter", "Google Cloud Storage", "Google Cloud VM", "IBM Cloud Object Storage", "IBM Cloud VM", "Oracle Cloud Object Storage", "Oracle Cloud VM", "DigitalOcean Droplet", "DigitalOcean Spaces", "Alibaba Cloud ECS", "Alibaba Cloud OSS", "Salesforce CRM", "Salesforce Marketing Cloud", "Adobe Creative Cloud", "Adobe Experience Manager", "Microsoft 365", "Microsoft Azure Blob Storage", "VMware Cloud on AWS", "Cisco Meraki MX", "Juniper vSRX", "HP Enterprise OneView", "Dell EMC PowerMax", "NetApp Cloud Volumes", "Red Hat OpenShift", "Kubernetes Cluster", "OpenStack Compute", "Ubuntu Server", "CentOS Virtual Machine", "SUSE Linux Enterprise Server", "Windows Server", "Mac Pro", "Android Virtual Device", "iOS Simulator", "IBM Watson Assistant", "Google Dialogflow", "Amazon Lex", "Microsoft Azure Bot Service", "Salesforce Einstein", "Oracle Digital Assistant", "Cisco Webex Teams", "Zoom Video Conferencing", "Slack Workspace", "Atlassian Jira", "GitHub Repository", "GitLab Project", "Bitbucket Repository", "Docker Container", "Kubernetes Pod", "Jenkins Server", "Ansible Playbook", "HashiCorp Vault", "Terraform Infrastructure", "Prometheus Monitoring", "Grafana Dashboard", "ELK Stack", "Splunk Enterprise", "Zabbix Server", "Nagios Core", "Datadog Monitoring", "New Relic APM", "Pingdom Website Monitoring", "AppDynamics Application Monitoring", "Dynatrace Platform", "Firebase Realtime Database", "MongoDB Atlas", "MySQL Database", "PostgreSQL Database", "Oracle Database", "Microsoft SQL Server", "Elasticsearch Cluster", "Redis Cache", "Memcached Cluster", "Apache Kafka", "RabbitMQ Messaging", "ActiveMQ Message Broker", "Amazon S3", "Azure Blob Storage", "Google Cloud BigQuery", "IBM Db2", "Salesforce Database", "Snowflake Data Warehouse", "Tableau Server", "Microsoft Power BI", "Qlik Sense", "Looker Analytics", "Adobe Analytics", "Google Analytics", "Facebook Pixel", "LinkedIn Insight Tag", "Twitter Pixel", "Amazon EC2 Instance", "Microsoft Azure Virtual Machine", "Google Cloud Compute Engine", "IBM Cloud Virtual Server", "Oracle Cloud Compute", "DigitalOcean Kubernetes", "Alibaba Cloud Elastic Compute", "Salesforce Platform", "SAP ERP", "ServiceNow ITSM", "Zendesk Support", "WordPress CMS", "Drupal CMS", "Joomla CMS", "Network Switch", "Server Rack", "Firewall Appliance", "Load Balancer", "Wireless Access Point", "Network Cable", "Power Distribution Unit", "Patch Panel", "UPS Battery Backup", "Network Router", "VoIP Phone", "Desktop Computer", "Laptop Computer", "Printer", "Scanner", "Projector", "Monitor", "Keyboard", "Mouse", "External Hard Drive", "USB Flash Drive", "Webcam", "Microphone", "Speakers", "Headphones", "Ethernet Adapter", "Wireless Mouse", "Wireless Keyboard", "Bluetooth Speaker", "Tablet", "Smartphone", "Digital Camera", "Video Camera", "Smart Watch", "Fitness Tracker", "GPS Navigator", "Drone", "Virtual Reality Headset", "Augmented Reality Glasses", "Home Security System", "Smart Thermostat", "Smart Lighting System", "Smart Lock", "Smart Doorbell", "Smart TV", "Streaming Media Player", "Game Console", "Gaming Controller", "Wireless Earbuds", "Bluetooth Earphones", "Portable Charger", "Power Bank", "Wireless Charger", "External DVD Drive", "CD/DVD Storage Case", "Router Bit Set", "Screwdriver Set", "Socket Wrench Set", "Pliers Set", "Hammer", "Drill", "Circular Saw", "Sanding Machine", "Paint Sprayer", "Air Compressor", "Welding Machine", "Generator", "Pressure Washer", "Lawn Mower", "Hedge Trimmer", "Leaf Blower", "Snow Blower", "Bicycle", "Helmet", "Golf Clubs", "Tennis Racket", "Basketball", "Football", "Baseball Glove", "Soccer Ball", "Volleyball", "Treadmill", "Exercise Bike", "Dumbbell Set", "Yoga Mat", "Resistance Bands", "Jump Rope", "Water Bottle", "Backpack", "Luggage Set", "Tent", "Sleeping Bag", "Camp Stove", "Cooler", "Hammock", "Fishing Rod", "Binoculars", "Telescope", "Picnic Blanket", "Guitar", "Keyboard Instrument", "Saxophone", "Trumpet", "Violin", "Microphone Stand", "Music Stand", "Amplifier", "Mixer", "Studio Monitor", "Music Production Software", "DJ Controller", "Paintbrush Set", "Canvas Set", "Easel", "Clay Sculpting Tools", "Pottery Wheel", "Knitting Needles", "Crochet Hooks", "Sewing Machine", "Fabric Cutting Mat", "Quilting Ruler", "Embroidery Hoop", "Cross-Stitch Kit", "Watercolor Paint Set", "Sketchbook", "Calligraphy Pen Set", "Origami Paper", "Puzzle", "Board Game", "Playing Cards", "Chess Set", "Rubik's Cube", "Magic Kit", "Toy Car", "Action Figure", "Dollhouse", "Stuffed Animal", "Building Blocks Set", "Play Kitchen", "Toy Tool Set", "Dress-Up Costume", "Puppet", "Bouncy Ball", "Bubble Solution", "Kite", "Remote Control Car", "Robot Toy", "Science Experiment Kit", "Telescope", "Microscope", "Chemistry Set", "Electronic Circuit Kit", "Coding Robot", "Educational Puzzle", "Artificial Intelligence Kit", "Solar-Powered Toy", "Indoor Plant", "Herb Garden Kit", "Bonsai Tree", "Terrarium", "Plant Stand", "Gardening Gloves", "Watering Can", "Pruning Shears", "Plant Mister", "Bird Feeder", "Wind Chimes", "Outdoor Bench", "Patio Umbrella", "BBQ Grill", "Fire Pit", "Outdoor Speakers", "Swimming Pool", "Hot Tub", "Outdoor Lighting", "Hammock", "Lawn Games Set", "Picnic Table", "Garden Shed", "Tool Shed", "Outdoor Storage Box", "Wheelbarrow", "Lawn Edger", "Garden Cart", "Rain Barrel", "Compost Bin", "Inflatable Kayak", "Camping Tent", "Folding Chair", "Outdoor Blanket", "Cooler", "Portable Grill", "Beach Umbrella", "Sunscreen", "Sunglasses", "Beach Towel", "Beach Ball", "Frisbee", "Water Slide", "Outdoor Volleyball Set", "Pool Float", "Outdoor Trampoline", "Badminton Set", "Basketball Hoop", "Slip 'n Slide", "Water Balloons", "Giant Jenga", "Outdoor Movie Projector", "Pet Carrier", "Dog Bed", "Cat Scratching Post", "Fish Tank", "Small Animal Cage", "Bird Cage", "Reptile Terrarium", "Hamster Wheel", "Pet Food Dispenser", "Pet Grooming Kit", "Pet Tracking Device", "Dog Training Collar", "Cat Litter Box", "Fish Food", "Small Animal Hay", "Bird Toys", "Reptile Heat Lamp", "Hamster Exercise Ball", "Pet Stroller", "Dog Leash", "Cat Collar", "Fish Aquarium Decorations", "Small Animal Hideout", "Bird Perch", "Reptile Thermometer", "Hamster Bedding", "Art Easel", "Watercolor Paper", "Paint Palette", "Brush Set", "Oil Paint Set", "Acrylic Paint Set", "Canvas Panel Set", "Sketching Pencils", "Charcoal Set", "Pastel Set", "Erasers", "Palette Knife Set", "Calligraphy Brushes", "Pottery Clay", "Pottery Tool Set", "Pottery Glazes", "Pottery Kiln", "Knitting Yarn", "Crochet Thread", "Sewing Thread", "Fabric Assortment", "Quilting Fabric", "Embroidery Floss", "Cross-Stitch Fabric", "Watercolor Brushes", "Drawing Pens", "Markers Set", "Colored Pencils", "Coloring Books", "Craft Glue", "Craft Scissors", "Craft Punches", "Stamps Set", "Heat Embossing Tool", "Cardstock Paper", "Origami Kit", "Jigsaw Puzzle", "Strategy Board Game", "Card Game", "Party Game", "Trivia Game", "Building Blocks", "Doll", "Toy Vehicle", "Puzzle Mat", "Magic Tricks Set", "Building Toy Set", "Kitchen Playset", "Tool Playset", "Costume Set", "Hand Puppet", "Bouncing Toy", "Bubble Wand", "Kite", "Remote Control Toy", "Robot Kit", "Science Kit", "Telescope", "Microscope", "Chemistry Set", "Electronic Kit", "Coding Toy", "Educational Toy", "AI Toy", "Solar Toy", "Indoor Plant", "Herb Garden", "Bonsai", "Terrarium", "Plant Stand", "Gardening Gloves", "Watering Can", "Pruning Shears", "Plant Mister", "Bird Feeder", "Wind Chimes", "Outdoor Bench", "Patio Umbrella", "BBQ Grill", "Fire Pit", "Outdoor Speakers", "Swimming Pool", "Hot Tub", "Outdoor Lighting", "Hammock", "Lawn Games Set", "Picnic Table", "Garden Shed", "Tool Shed", "Outdoor Storage Box", "Wheelbarrow", "Lawn Edger", "Garden Cart", "Rain Barrel", "Compost Bin", "Inflatable Kayak", "Camping Tent", "Folding Chair", "Outdoor Blanket", "Cooler", "Portable Grill", "Beach Umbrella", "Sunscreen", "Sunglasses", "Beach Towel", "Beach Ball", "Frisbee", "Water Slide", "Outdoor Volleyball Set", "Pool Float", "Outdoor Trampoline", "Badminton Set", "Basketball Hoop", "Slip 'n Slide", "Water Balloons", "Giant Jenga", "Outdoor Movie Projector", "Pet Carrier", "Dog Bed", "Cat Scratching Post", "Fish Tank", "Small Animal Cage", "Bird Cage", "Reptile Terrarium", "Hamster Wheel", "Pet Food Dispenser", "Pet Grooming Kit", "Pet Tracking Device", "Dog Training Collar", "Cat Litter Box", "Fish Food", "Small Animal Hay", "Bird Toys", "Reptile Heat Lamp", "Hamster Exercise Ball", "Pet Stroller", "Dog Leash", "Cat Collar", "Fish Aquarium Decorations", "Small Animal Hideout", "Bird Perch", "Reptile Thermometer", "Hamster Bedding", "Art Easel", "Watercolor Paper", "Paint Palette", "Brush Set", "Oil Paint Set", "Acrylic Paint Set", "Canvas Panel Set", "Sketching Pencils", "Charcoal Set", "Pastel Set", "Erasers", "Palette Knife Set", "Calligraphy Brushes", "Pottery Clay", "Pottery Tool Set", "Pottery Glazes", "Pottery Kiln", "Knitting Yarn", "Crochet Thread", "Sewing Thread", "Fabric Assortment", "Quilting Fabric", "Embroidery Floss", "Cross-Stitch Fabric", "Watercolor Brushes", "Drawing Pens", "Markers Set", "Colored Pencils", "Coloring Books", "Craft Glue", "Craft Scissors", "Craft Punches", "Stamps Set", "Heat Embossing Tool", "Cardstock Paper", "Origami Kit", "Jigsaw Puzzle", "Strategy Board Game", "Card Game", "Party Game", "Trivia Game", "Building Blocks", "Doll", "Toy Vehicle", "Puzzle Mat", "Magic Tricks Set", "Building Toy Set", "Kitchen Playset", "Tool Playset", "Costume Set", "Hand Puppet", "Bouncing Toy", "Bubble Wand", "Kite", "Remote Control Toy", "Robot Kit", "Science Kit", "Telescope", "Microscope", "Chemistry Set", "Electronic Kit", "Coding Toy", "Educational Toy", "AI Toy", "Solar Toy", "Indoor Plant", "Herb Garden", "Bonsai", "Terrarium", "Plant Stand", "Gardening Gloves", "Watering Can", "Pruning Shears", "Plant Mister", "Bird Feeder", "Wind Chimes", "Outdoor Bench", "Patio Umbrella", "BBQ Grill", "Fire Pit", "Outdoor Speakers", "Swimming Pool", "Hot Tub", "Outdoor Lighting", "Hammock", "Lawn Games Set", "Picnic Table", "Garden Shed", "Tool Shed", "Outdoor Storage Box", "Wheelbarrow", "Lawn Edger", "Garden Cart", "Rain Barrel", "Compost Bin", "Inflatable Kayak", "Camping Tent", "Folding Chair", "Outdoor Blanket", "Cooler", "Portable Grill", "Beach Umbrella", "Sunscreen", "Sunglasses", "Beach Towel", "Beach Ball", "Frisbee", "Water Slide", "Outdoor Volleyball Set", "Pool Float", "Outdoor Trampoline", "Badminton Set", "Basketball Hoop", "Slip 'n Slide", "Water Balloons", "Giant Jenga", "Outdoor Movie Projector", "Pet Carrier", "Dog Bed", "Cat Scratching Post", "Fish Tank", "Small Animal Cage", "Bird Cage", "Reptile Terrarium", "Hamster Wheel", "Pet Food Dispenser", "Pet Grooming Kit", "Pet Tracking Device", "Dog Training Collar", "Cat Litter Box", "Fish Food", "Small Animal Hay", "Bird Toys", "Reptile Heat Lamp", "Hamster Exercise Ball", "Pet Stroller", "Dog Leash", "Cat Collar", "Fish Aquarium Decorations", "Small Animal Hideout", "Bird Perch", "Reptile Thermometer", "Hamster Bedding", "Art Easel", "Watercolor Paper", "Paint Palette", "Brush Set", "Oil Paint Set", "Acrylic Paint Set", "Canvas Panel Set", "Sketching Pencils", "Charcoal Set", "Pastel Set", "Erasers", "Palette Knife Set", "Calligraphy Brushes", "Pottery Clay", "Pottery Tool Set", "Pottery Glazes", "Pottery Kiln", "Knitting Yarn", "Crochet Thread", "Sewing Thread", "Fabric Assortment", "Quilting Fabric", "Embroidery Floss", "Cross-Stitch Fabric", "Watercolor Brushes", "Drawing Pens", "Markers Set", "Colored Pencils", "Coloring Books", "Craft Glue", "Craft Scissors", "Craft Punches", "Stamps Set", "Heat Embossing Tool", "Cardstock Paper", "Origami Kit", "Jigsaw Puzzle", "Strategy Board Game", "Card Game", "Party Game", "Trivia Game", "Building Blocks", "Doll", "Toy Vehicle", "Puzzle Mat", "Magic Tricks Set", "Building Toy Set", "Kitchen Playset", "Tool Playset", "Costume Set", "Hand Puppet", "Bouncing Toy", "Bubble Wand", "Kite", "Remote Control Toy", "Robot Kit", "Science Kit", "Telescope", "Microscope", "Chemistry Set", "Electronic Kit", "Coding Toy", "Educational Toy", "AI Toy", "Solar Toy", "Indoor Plant", "Herb Garden", "Bonsai", "Terrarium", "Plant Stand", "Gardening Gloves", "Watering Can", "Pruning Shears", "Plant Mister", "Bird Feeder", "Wind Chimes", "Outdoor Bench", "Patio Umbrella", "BBQ Grill", "Fire Pit", "Outdoor Speakers", "Swimming Pool", "Hot Tub", "Outdoor Lighting", "Hammock", "Lawn Games Set", "Picnic Table", "Garden Shed", "Tool Shed", "Outdoor Storage Box", "Wheelbarrow", "Lawn Edger", "Garden Cart", "Rain Barrel", "Compost Bin", "Inflatable Kayak", "Camping Tent", "Folding Chair", "Outdoor Blanket", "Cooler", "Portable Grill", "Beach Umbrella", "Sunscreen", "Sunglasses", "Beach Towel", "Beach Ball", "Frisbee", "Water Slide", "Outdoor Volleyball Set", "Pool Float", "Outdoor Trampoline", "Badminton Set", "Basketball Hoop", "Slip 'n Slide", "Water Balloons", "Giant Jenga", "Outdoor Movie Projector", "Pet Carrier", "Dog Bed", "Cat Scratching Post", "Fish Tank", "Small Animal Cage", "Bird Cage", "Reptile Terrarium", "Hamster Wheel", "Pet Food Dispenser", "Pet Grooming Kit", "Pet Tracking Device", "Dog Training Collar", "Cat Litter Box", "Fish Food", "Small Animal Hay", "Bird Toys", "Reptile Heat Lamp", "Hamster Exercise Ball", "Pet Stroller", "Dog Leash", "Cat Collar", "Fish Aquarium Decorations", "Small Animal Hideout", "Bird Perch", "Reptile Thermometer", "Hamster Bedding", "Art Easel", "Watercolor Paper", "Paint Palette", "Brush Set", "Oil Paint Set", "Acrylic Paint Set", "Canvas Panel Set", "Sketching Pencils", "Charcoal Set", "Pastel Set", "Erasers", "Palette Knife Set", "Calligraphy Brushes", "Pottery Clay", "Pottery Tool Set", "Pottery Glazes", "Pottery Kiln", "Knitting Yarn", "Crochet Thread", "Sewing Thread", "Fabric Assortment", "Quilting Fabric", "Embroidery Floss", "Cross-Stitch Fabric", "Watercolor Brushes", "Drawing Pens", "Markers Set", "Colored Pencils", "Coloring Books", "Craft Glue", "Craft Scissors", "Craft Punches", "Stamps Set", "Heat Embossing Tool", "Cardstock Paper", "Origami Kit", "Jigsaw Puzzle", "Strategy Board Game", "Card Game", "Party Game", "Trivia Game", "Building Blocks", "Doll", "Toy Vehicle", "Puzzle Mat", "Magic Tricks Set", "Building Toy Set", "Kitchen Playset", "Tool Playset", "Costume Set", "Hand Puppet", "Bouncing Toy", "Bubble Wand", "Kite", "Remote Control Toy", "Robot Kit", "Science Kit", "Telescope", "Microscope", "Chemistry Set", "Electronic Kit", "Coding Toy", "Educational Toy", "AI Toy", "Solar Toy", "Indoor Plant", "Herb Garden", "Bonsai", "Terrarium", "Plant Stand", "Gardening Gloves", "Watering Can", "Pruning Shears", "Plant Mister", "Bird Feeder", "Wind Chimes", "Outdoor Bench", "Patio Umbrella", "BBQ Grill", "Fire Pit", "Outdoor Speakers", "Swimming Pool", "Hot Tub", "Outdoor Lighting", "Hammock", "Lawn Games Set", "Picnic Table", "Garden Shed", "Tool Shed", "Outdoor Storage Box", "Wheelbarrow", "Lawn Edger", "Garden Cart", "Rain Barrel", "Compost Bin", "Inflatable Kayak", "Camping Tent", "Folding Chair", "Outdoor Blanket", "Cooler", "Portable Grill", "Beach Umbrella", "Sunscreen", "Sunglasses", "Beach Towel"],
	'group': ['Incident Team', 'Major Incident Team', 'Service Request Fulfillment Team', 'Problem Management Team', 'Change Team', 'Release Team', 'Database Team', 'Hardware Team', 'Capacity Management Team', 'Supplier Management Team', 'Service Design Team', 'Software Team', 'Network Team', 'Helpdesk Monitoring Team'],
	'locations': ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming", "Alberta", "British Columbia", "Manitoba", "New Brunswick", "Newfoundland and Labrador", "Nova Scotia", "Ontario", "Prince Edward Island", "Quebec", "Saskatchewan", "Northwest Territories", "Nunavut", "Yukon", "Argentina", "Bolivia", "Brazil", "Chile", "Colombia", "Ecuador", "Guyana", "Paraguay", "Peru", "Suriname", "Uruguay", "Venezuela", "Australia", "Fiji", "Kiribati", "Marshall Islands", "Micronesia", "Nauru", "New Zealand", "Palau", "Papua New Guinea", "Samoa", "Solomon Islands", "Tonga", "Tuvalu", "Vanuatu", "Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Norway", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "United Kingdom"],
	'name': ["john", "mary", "alex", "emma", "sam", "olivia", "david", "sophia", "michael", "lucy", "james", "ava", "benjamin", "chloe", "william", "emily", "daniel", "mia", "matthew", "harper"]
}

IMPACTS = ['low', 'medium', 'high']
FIELD_TYPES = ['text', 'paragraph', 'number', 'integer', 'dropdown', 'date', 'checkbox', 'bigint']
SEED = {
	'locations': 13178,
	'departments': 114260,
	'asset_types': 1,
	'requesters': 10000,
	'agents': 7000,
	'assets': 10000,
	'groups': 2030,
	'type_fields': 100
}

class DBConnection(object):
	
	def __enter__(self, *args, **kwargs):
		self.connection = pymysql.connect(
			**MYSQL_CONFIG,
			charset='utf8mb4',
			cursorclass=pymysql.cursors.DictCursor
		)
		self.cursor = self.connection.cursor()
		return self
	
	def __exit__(self, *args, **kwargs):
		self.connection and self.connection.close()
		self.cursor and self.cursor.close()
		
	def commit(self):
		self.cursor.commit()


# In[368]:


class CustomORM(object):
	def __new__(self, *args, **kwargs):
		table_name, fetch_type = (args[0], args[1])
		data = []
		col = args[2] if len(args) >= 3 else '*'
		with DBConnection() as db:
			print(f'Fetching::{table_name}::{fetch_type}::{col}')
			q = f'select {col} from `{table_name}` where account_id = {ACCOUNT_ID}'
			if fetch_type == 'last':
				q += f' order by {col} desc limit 1;'
			print('Executing Query\n\t{q}'.format(q = q))
			db.cursor.execute(q)
			data = db.cursor.fetchall()
			data = list(map(lambda x : x[col], data))
			if fetch_type == 'last' and len(data) > 0:
				data = data[0]
		return data


# In[394]:

def get_table_fields(table_name):
	with DBConnection() as db:
		db.cursor.execute(f'describe {table_name}')
		return list(map(lambda x : (x['Field'], x['Null']), db.cursor.fetchall()))

def get_random_string(_type, size = 10):
	if _type == 'number':
		return ''.join([secrets.choice(string.digits) for _ in range(size)])
	elif _type == 'alpha':
		characters = string.ascii_letters + string.digits
		return ''.join(secrets.choice(characters) for _ in range(size))
	elif _type == 'name':
		return random.choice(ENTITY_MAP['name']) + get_random_string("number", 5)
	else:
		return f'{random.choice(ENTITY_MAP[_type])}-{get_random_string("alpha", 5)}'

def get_cur_date():
	return datetime.now().isoformat()

def get_ref_id(ref_id):
	ref_id = str(ref_id)[0:3] + ''.join([random.choice(string.digits) for _ in range(12)])
	if ref_id in REF_ID_MAP:
		print('Got duplicate REF iD')
		return get_ref_id()
	REF_ID_MAP[ref_id] = True
	return ref_id

# In[402]:

class SeedData:
	def __init__(self, db):
		self.db = db
		
	def execute_query(self, query, params=None):
		self.db.cursor.execute(query, params)
		if params is None: return self.db.cursor.fetchall()
		
	def execute_many(self, query, params):
		self.db.cursor.executemany(query, params)
		self.db.connection.commit()
		
	def fetch_required_meta(self):
		self.asset_states = CustomORM('cmdb_ci_type_field_values', 'list', 'name')
		self.vendor_ids = CustomORM('itil_vendors', 'list', 'id')
		self.product_ids = CustomORM('itil_products', 'list', 'id')
		self.department_ids = CustomORM('itil_departments', 'list', 'id')
		self.location_ids = CustomORM('cmdb_locations', 'list', 'id')
		self.ref_ids = CustomORM('cmdb_ci_types', 'list', 'ref_id')
		self.group_ids = CustomORM('groups', 'list', 'id')
		
		query = f"""
		select id from cmdb_ci_types where ref_id = {REF_ID} and account_id = {ACCOUNT_ID}
		"""
		self.ci_type_ids = list(map(lambda x : x['id'], self.execute_query(query)))

	def seed_entity_data(self):
		self.fetch_required_meta()        
		self.seed_asset_types()
		self.seed_users(False)
		self.seed_departments()
		self.seed_locations()
		self.seed_users(True)
		self.seed_agent_groups()
		self.seed_type_fields()
		self.fetch_required_meta()
		self.seed_assets()
		
	def seed_asset_types(self):
		query = """
		INSERT INTO cmdb_ci_types (
			name, description, parent_ci_type, is_default, account_id, ancestry, ancestry_depth, 
			created_at, updated_at, ref_id, label, disabled
		)
		VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
		"""
		params = []
		for _ in range(SEED['asset_types']):
			params.append([
				get_random_string('asset'),
				"",
				self.ci_type_ids[0],
				False,
				ACCOUNT_ID,
				self.ci_type_ids[0],
				1,
				get_cur_date(),
				get_cur_date(),
				get_ref_id(REF_ID),
				get_random_string('asset'),
				0
			])
		self.execute_many(query, params)

	def seed_locations(self):
		query = """
		INSERT INTO cmdb_locations (name, account_id)
		VALUES (%s, %s)
		"""
		params = []
		for _ in range(SEED['locations']):
			params.append([
				get_random_string('locations'),  # name
				ACCOUNT_ID
			])
		self.execute_many(query, params)
		
	def seed_departments(self):
		query = """
		INSERT INTO itil_departments (name, description, account_id, head_id, prime_user_id, created_at, updated_at)
		VALUES (%s, %s, %s, %s, %s, %s, %s)
		"""
		params = []
		for _ in range(SEED['departments']):
			params.append([
				get_random_string('department'),
				"",
				ACCOUNT_ID,
				None,
				None,
				get_cur_date(),
				get_cur_date()
			])
		self.execute_many(query, params)
		
	def seed_users(self, is_agent = False):
		query = """
		INSERT INTO users (name, email, persistence_token, login_count, failed_login_count, account_id, active, delta, helpdesk_agent, created_at, updated_at) 
		VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
		"""
		params = []
		for _ in range(SEED['agents'] if is_agent else SEED['requesters']):
			params.append([
				get_random_string('name'),
				get_random_string('name') + '@gmail.com',
				"",
				0, 
				0,
				ACCOUNT_ID,
				1,
				0,
				1 if is_agent else 0,
				get_cur_date(),
				get_cur_date()
			])
		last_user_id = CustomORM('users', 'last', 'id')
		self.execute_many(query, params)
		if is_agent:
			self.seed_agents(last_user_id)

	def seed_agents(self, last_user_id):
		query = """
		INSERT INTO agents (user_id, account_id, available, license_type) values (%s, %s, %s, %s);
		"""
		params = []
		for _ in range(SEED['agents']):
			last_user_id += 1
			params.append([
				last_user_id,
				ACCOUNT_ID,
				1,
				1
			])
		self.execute_many(query, params)
   
	def seed_agent_groups(self):
		query = """
		INSERT INTO `groups` (name, description, account_id, type, business_function, workspace_id) 
		VALUES (%s, %s, %s, %s, %s, %s);
		"""
		params = []
		for _ in range(SEED['groups']):
			params.append([
				get_random_string('group'),
				"",
				ACCOUNT_ID,
				'Group',
				0,
				WORKSPACE_ID
			])
		self.execute_many(query, params)

	def seed_type_fields(self):
		ci_type_id = self.ci_type_ids[0]
		last_position = CustomORM('cmdb_ci_type_fields', 'last', 'position')
		query = """
		INSERT INTO cmdb_ci_type_fields (
			name, label, active, field_type, position, required, table_name, column_name, ci_type_id, account_id, 
			col_type, deleted, is_unique
		) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
		"""
		params = []
		for _ in range(SEED['type_fields']):
			last_position += 1
			field_type = random.choice(FIELD_TYPES)
			params.append([
				f'{field_type}_{get_random_string("alpha")}',
				get_random_string('asset'),
				1,
				f'custom_{field_type}',
				last_position,
				0,
				'cmdb_ci_level_0_fields',
				'',
				ci_type_id, ACCOUNT_ID, field_type, 0, 0
			])
		self.execute_many(query, params)
		

	def seed_assets(self):
		query = """
		INSERT INTO cmdb_config_items (name, ci_type_id, impact, account_id, department_id, group_id, display_id, location_id, roots_ref_id)
		VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
		"""
		params = []
		for _ in range(SEED['assets']):
			params.append([
				get_random_string('asset'),  # name
				self.ci_type_ids[0],  # ci_type_id
				IMPACTS.index(random.choice(IMPACTS)),  # impact
				ACCOUNT_ID,
				random.choice(self.department_ids),
				random.choice(self.group_ids),
				get_random_string('number'),
				random.choice(self.location_ids),
				REF_ID
			])
		self.execute_many(query, params)
		
with DBConnection() as db:
	seeder = SeedData(db)
	seeder.seed_entity_data()


OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 61] Connection refused)")